<a href="https://colab.research.google.com/github/Pluto2/pytorch/blob/main/uformer_dcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install python==3.7
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -U openmim
!mim install mmcv-full==1.5.0
!pip install timm==0.6.11 mmdet==2.28.1
!pip install opencv-python termcolor yacs pyyaml scipy
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement python==3.7 (from versions: none)
ERROR: No matching distribution found for python==3.7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 KB 658.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 525.0 kB/s eta 0:00:00
ERROR: Operation cancelled by user
/bin/bash: mim: command not found
Looking in indexes: https://pypi.org/simple, https://us-p

In [ ]:
!pip install python==3.7
!sh make.sh

import DCNv3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement python==3.7 (from versions: none)
ERROR: No matching distribution found for python==3.7
running build
running build_ext
/usr/local/lib/python3.9/dist-packages/torch/utils/cpp_extension.py:387: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
running install
/usr/local/lib/python3.9/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bd

ImportError: ignored

In [ ]:
import sys

egg_path='/usr/local/lib/python3.9/dist-packages/DCNv3-1.0-py3.9-linux-x86_64.egg'
sys.path.append(egg_path)

import DCNv3

-I/usr/include/python3.8 -I/usr/include/python3.8  -Wno-unused-result -Wsign-compare -g -fdebug-prefix-map=/build/python3.8-B32VZ9/python3.8-3.8.10=. -specs=/usr/share/dpkg/no-pie-compile.specs -fstack-protector -Wformat -Werror=format-security  -DNDEBUG -g -fwrapv -O3 -Wall
-L/usr/lib/python3.8/config-3.8-x86_64-linux-gnu -L/usr/lib  -lcrypt -lpthread -ldl  -lutil -lm -lm 


ImportError: ignored

In [ ]:
import torch
import torch.nn as nn
import torch.utils.checkpoint as checkpoint
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
import torch.nn.functional as F
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
import math
import numpy as np
import time
from torch import einsum
# from __future__ import absolute_import
# from __future__ import print_function
# from __future__ import division

import warnings
from torch import nn
import torch.nn.functional as F
from torch.nn.init import xavier_uniform_, constant_
# from ..functions import DCNv3Function, dcnv3_core_pytorch
from torch.autograd import Function
from torch.autograd.function import once_differentiable
from torch.cuda.amp import custom_bwd, custom_fwd
'''
Intern_Image里面的代码
---------------Begin-----------------
'''
class to_channels_first(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.permute(0, 3, 1, 2)


class to_channels_last(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x.permute(0, 2, 3, 1)


def build_norm_layer(dim,
                     norm_layer,
                     in_format='channels_last',
                     out_format='channels_last',
                     eps=1e-6):
    layers = []
    if norm_layer == 'BN':
        if in_format == 'channels_last':
            layers.append(to_channels_first())
        layers.append(nn.BatchNorm2d(dim))
        if out_format == 'channels_last':
            layers.append(to_channels_last())
    elif norm_layer == 'LN':
        if in_format == 'channels_first':
            layers.append(to_channels_last())
        layers.append(nn.LayerNorm(dim, eps=eps))
        if out_format == 'channels_first':
            layers.append(to_channels_first())
    else:
        raise NotImplementedError(
            f'build_norm_layer does not support {norm_layer}')
    return nn.Sequential(*layers)


def build_act_layer(act_layer):
    if act_layer == 'ReLU':
        return nn.ReLU(inplace=True)
    elif act_layer == 'SiLU':
        return nn.SiLU(inplace=True)
    elif act_layer == 'GELU':
        return nn.GELU()

    raise NotImplementedError(f'build_act_layer does not support {act_layer}')


In [ ]:
# import DCNv3

class DCNv3Function(Function):
    @staticmethod
    @custom_fwd
    def forward(
            ctx, input, offset, mask,
            kernel_h, kernel_w, stride_h, stride_w,
            pad_h, pad_w, dilation_h, dilation_w,
            group, group_channels, offset_scale, im2col_step):
        ctx.kernel_h = kernel_h
        ctx.kernel_w = kernel_w
        ctx.stride_h = stride_h
        ctx.stride_w = stride_w
        ctx.pad_h = pad_h
        ctx.pad_w = pad_w
        ctx.dilation_h = dilation_h
        ctx.dilation_w = dilation_w
        ctx.group = group
        ctx.group_channels = group_channels
        ctx.offset_scale = offset_scale
        ctx.im2col_step = im2col_step
        output = DCNv3.dcnv3_forward(
            input, offset, mask, kernel_h,
            kernel_w, stride_h, stride_w, pad_h,
            pad_w, dilation_h, dilation_w, group,
            group_channels, offset_scale, ctx.im2col_step)
        ctx.save_for_backward(input, offset, mask)

        return output

    @staticmethod
    @once_differentiable
    @custom_bwd
    def backward(ctx, grad_output):
        input, offset, mask = ctx.saved_tensors
        grad_input, grad_offset, grad_mask = \
            DCNv3.dcnv3_backward(
                input, offset, mask, ctx.kernel_h,
                ctx.kernel_w, ctx.stride_h, ctx.stride_w, ctx.pad_h,
                ctx.pad_w, ctx.dilation_h, ctx.dilation_w, ctx.group,
                ctx.group_channels, ctx.offset_scale, grad_output.contiguous(), ctx.im2col_step)

        return grad_input, grad_offset, grad_mask, \
            None, None, None, None, None, None, None, None, None, None, None, None

    @staticmethod
    def symbolic(g, input, offset, mask, kernel_h, kernel_w, stride_h,
                 stride_w, pad_h, pad_w, dilation_h, dilation_w, group,
                 group_channels, offset_scale, im2col_step):
        """Symbolic function for mmdeploy::DCNv3.

        Returns:
            DCNv3 op for onnx.
        """
        return g.op(
            'mmdeploy::TRTDCNv3',
            input,
            offset,
            mask,
            kernel_h_i=int(kernel_h),
            kernel_w_i=int(kernel_w),
            stride_h_i=int(stride_h),
            stride_w_i=int(stride_w),
            pad_h_i=int(pad_h),
            pad_w_i=int(pad_w),
            dilation_h_i=int(dilation_h),
            dilation_w_i=int(dilation_w),
            group_i=int(group),
            group_channels_i=int(group_channels),
            offset_scale_f=float(offset_scale),
            im2col_step_i=int(im2col_step),
        )


def _get_reference_points(spatial_shapes, device, kernel_h, kernel_w, dilation_h, dilation_w, pad_h=0, pad_w=0, stride_h=1, stride_w=1):
    _, H_, W_, _ = spatial_shapes
    H_out = (H_ - (dilation_h * (kernel_h - 1) + 1)) // stride_h + 1
    W_out = (W_ - (dilation_w * (kernel_w - 1) + 1)) // stride_w + 1

    ref_y, ref_x = torch.meshgrid(
        torch.linspace(
            # pad_h + 0.5,
            # H_ - pad_h - 0.5,
            (dilation_h * (kernel_h - 1)) // 2 + 0.5,
            (dilation_h * (kernel_h - 1)) // 2 + 0.5 + (H_out - 1) * stride_h,
            H_out,
            dtype=torch.float32,
            device=device),
        torch.linspace(
            # pad_w + 0.5,
            # W_ - pad_w - 0.5,
            (dilation_w * (kernel_w - 1)) // 2 + 0.5,
            (dilation_w * (kernel_w - 1)) // 2 + 0.5 + (W_out - 1) * stride_w,
            W_out,
            dtype=torch.float32,
            device=device))
    ref_y = ref_y.reshape(-1)[None] / H_
    ref_x = ref_x.reshape(-1)[None] / W_

    ref = torch.stack((ref_x, ref_y), -1).reshape(
        1, H_out, W_out, 1, 2)

    return ref


def _generate_dilation_grids(spatial_shapes, kernel_h, kernel_w, dilation_h, dilation_w, group, device):
    _, H_, W_, _ = spatial_shapes
    points_list = []
    x, y = torch.meshgrid(
        torch.linspace(
            -((dilation_w * (kernel_w - 1)) // 2),
            -((dilation_w * (kernel_w - 1)) // 2) +
            (kernel_w - 1) * dilation_w, kernel_w,
            dtype=torch.float32,
            device=device),
        torch.linspace(
            -((dilation_h * (kernel_h - 1)) // 2),
            -((dilation_h * (kernel_h - 1)) // 2) +
            (kernel_h - 1) * dilation_h, kernel_h,
            dtype=torch.float32,
            device=device))

    points_list.extend([x / W_, y / H_])
    grid = torch.stack(points_list, -1).reshape(-1, 1, 2).\
        repeat(1, group, 1).permute(1, 0, 2)
    grid = grid.reshape(1, 1, 1, group * kernel_h * kernel_w, 2)

    return grid


def dcnv3_core_pytorch(
        input, offset, mask, kernel_h,
        kernel_w, stride_h, stride_w, pad_h,
        pad_w, dilation_h, dilation_w, group,
        group_channels, offset_scale):
    # for debug and test only,
    # need to use cuda version instead
    input = F.pad(
        input,
        [0, 0, pad_h, pad_h, pad_w, pad_w])
    N_, H_in, W_in, _ = input.shape
    _, H_out, W_out, _ = offset.shape

    ref = _get_reference_points(
        input.shape, input.device, kernel_h, kernel_w, dilation_h, dilation_w, pad_h, pad_w, stride_h, stride_w)
    grid = _generate_dilation_grids(
        input.shape, kernel_h, kernel_w, dilation_h, dilation_w, group, input.device)
    spatial_norm = torch.tensor([W_in, H_in]).reshape(1, 1, 1, 2).\
        repeat(1, 1, 1, group*kernel_h*kernel_w).to(input.device)

    sampling_locations = (ref + grid * offset_scale).repeat(N_, 1, 1, 1, 1).flatten(3, 4) + \
        offset * offset_scale / spatial_norm

    P_ = kernel_h * kernel_w
    sampling_grids = 2 * sampling_locations - 1
    # N_, H_in, W_in, group*group_channels -> N_, H_in*W_in, group*group_channels -> N_, group*group_channels, H_in*W_in -> N_*group, group_channels, H_in, W_in
    input_ = input.view(N_, H_in*W_in, group*group_channels).transpose(1, 2).\
        reshape(N_*group, group_channels, H_in, W_in)
    # N_, H_out, W_out, group*P_*2 -> N_, H_out*W_out, group, P_, 2 -> N_, group, H_out*W_out, P_, 2 -> N_*group, H_out*W_out, P_, 2
    sampling_grid_ = sampling_grids.view(N_, H_out*W_out, group, P_, 2).transpose(1, 2).\
        flatten(0, 1)
    # N_*group, group_channels, H_out*W_out, P_
    sampling_input_ = F.grid_sample(
        input_, sampling_grid_, mode='bilinear', padding_mode='zeros', align_corners=False)

    # (N_, H_out, W_out, group*P_) -> N_, H_out*W_out, group, P_ -> (N_, group, H_out*W_out, P_) -> (N_*group, 1, H_out*W_out, P_)
    mask = mask.view(N_, H_out*W_out, group, P_).transpose(1, 2).\
        reshape(N_*group, 1, H_out*W_out, P_)
    output = (sampling_input_ * mask).sum(-1).view(N_,
                                                   group*group_channels, H_out*W_out)

    return output.transpose(1, 2).reshape(N_, H_out, W_out, -1).contiguous()

"""
代替 from ops_dcnv3 import modules as opsm
这里直接复制 ops_dcnv3/modules
"""
# --------------------------------------------------------
# InternImage
# Copyright (c) 2022 OpenGVLab
# Licensed under The MIT License [see LICENSE for details]
# --------------------------------------------------------
def _is_power_of_2(n):
    if (not isinstance(n, int)) or (n < 0):
        raise ValueError(
            "invalid input for _is_power_of_2: {} (type: {})".format(n, type(n)))

    return (n & (n - 1) == 0) and n != 0


class CenterFeatureScaleModule(nn.Module):
    def forward(self,
                query,
                center_feature_scale_proj_weight,
                center_feature_scale_proj_bias):
        center_feature_scale = F.linear(query,
                                        weight=center_feature_scale_proj_weight,
                                        bias=center_feature_scale_proj_bias).sigmoid()
        return center_feature_scale


class DCNv3_pytorch(nn.Module):
    def __init__(
            self,
            channels=64,
            kernel_size=3,
            dw_kernel_size=None,
            stride=1,
            pad=1,
            dilation=1,
            group=4,
            offset_scale=1.0,
            act_layer='GELU',
            norm_layer='LN',
            center_feature_scale=False):
        """
        DCNv3 Module
        :param channels
        :param kernel_size
        :param stride
        :param pad
        :param dilation
        :param group
        :param offset_scale
        :param act_layer
        :param norm_layer
        """
        super().__init__()
        if channels % group != 0:
            raise ValueError(
                f'channels must be divisible by group, but got {channels} and {group}')
        _d_per_group = channels // group
        dw_kernel_size = dw_kernel_size if dw_kernel_size is not None else kernel_size
        # you'd better set _d_per_group to a power of 2 which is more efficient in our CUDA implementation
        if not _is_power_of_2(_d_per_group):
            warnings.warn(
                "You'd better set channels in DCNv3 to make the dimension of each attention head a power of 2 "
                "which is more efficient in our CUDA implementation.")

        self.offset_scale = offset_scale
        self.channels = channels
        self.kernel_size = kernel_size
        self.dw_kernel_size = dw_kernel_size
        self.stride = stride
        self.dilation = dilation
        self.pad = pad
        self.group = group
        self.group_channels = channels // group
        self.offset_scale = offset_scale
        self.center_feature_scale = center_feature_scale

        self.dw_conv = nn.Sequential(
            nn.Conv2d(
                channels,
                channels,
                kernel_size=dw_kernel_size,
                stride=1,
                padding=(dw_kernel_size - 1) // 2,
                groups=channels),
            build_norm_layer(
                channels,
                norm_layer,
                'channels_first',
                'channels_last'),
            build_act_layer(act_layer))
        self.offset = nn.Linear(
            channels,
            group * kernel_size * kernel_size * 2)
        self.mask = nn.Linear(
            channels,
            group * kernel_size * kernel_size)
        self.input_proj = nn.Linear(channels, channels)
        self.output_proj = nn.Linear(channels, channels)
        self._reset_parameters()

        if center_feature_scale:
            self.center_feature_scale_proj_weight = nn.Parameter(
                torch.zeros((group, channels), dtype=torch.float))
            self.center_feature_scale_proj_bias = nn.Parameter(
                torch.tensor(0.0, dtype=torch.float).view((1,)).repeat(group, ))
            self.center_feature_scale_module = CenterFeatureScaleModule()

    def _reset_parameters(self):
        constant_(self.offset.weight.data, 0.)
        constant_(self.offset.bias.data, 0.)
        constant_(self.mask.weight.data, 0.)
        constant_(self.mask.bias.data, 0.)
        xavier_uniform_(self.input_proj.weight.data)
        constant_(self.input_proj.bias.data, 0.)
        xavier_uniform_(self.output_proj.weight.data)
        constant_(self.output_proj.bias.data, 0.)

    def forward(self, input):
        """
        :param query                       (N, H, W, C)
        :return output                     (N, H, W, C)
        """
        N, H, W, _ = input.shape

        x = self.input_proj(input)
        x_proj = x

        x1 = input.permute(0, 3, 1, 2)
        x1 = self.dw_conv(x1)
        offset = self.offset(x1)
        mask = self.mask(x1).reshape(N, H, W, self.group, -1)
        mask = F.softmax(mask, -1).reshape(N, H, W, -1)

        x = dcnv3_core_pytorch(
            x, offset, mask,
            self.kernel_size, self.kernel_size,
            self.stride, self.stride,
            self.pad, self.pad,
            self.dilation, self.dilation,
            self.group, self.group_channels,
            self.offset_scale)
        if self.center_feature_scale:
            center_feature_scale = self.center_feature_scale_module(
                x1, self.center_feature_scale_proj_weight, self.center_feature_scale_proj_bias)
            # N, H, W, groups -> N, H, W, groups, 1 -> N, H, W, groups, _d_per_group -> N, H, W, channels
            center_feature_scale = center_feature_scale[..., None].repeat(
                1, 1, 1, 1, self.channels // self.group).flatten(-2)
            x = x * (1 - center_feature_scale) + x_proj * center_feature_scale
        x = self.output_proj(x)

        return x


class DCNv3(nn.Module):
    def __init__(
            self,
            channels=64,
            kernel_size=3,
            dw_kernel_size=None,
            stride=1,
            pad=1,
            dilation=1,
            group=4,
            offset_scale=1.0,
            act_layer='GELU',
            norm_layer='LN',
            center_feature_scale=False):
        """
        DCNv3 Module
        :param channels
        :param kernel_size
        :param stride
        :param pad
        :param dilation
        :param group
        :param offset_scale
        :param act_layer
        :param norm_layer
        """
        super().__init__()
        if channels % group != 0:
            raise ValueError(
                f'channels must be divisible by group, but got {channels} and {group}')
        _d_per_group = channels // group
        dw_kernel_size = dw_kernel_size if dw_kernel_size is not None else kernel_size
        # you'd better set _d_per_group to a power of 2 which is more efficient in our CUDA implementation
        if not _is_power_of_2(_d_per_group):
            warnings.warn(
                "You'd better set channels in DCNv3 to make the dimension of each attention head a power of 2 "
                "which is more efficient in our CUDA implementation.")

        self.offset_scale = offset_scale
        self.channels = channels
        self.kernel_size = kernel_size
        self.dw_kernel_size = dw_kernel_size
        self.stride = stride
        self.dilation = dilation
        self.pad = pad
        self.group = group
        self.group_channels = channels // group
        self.offset_scale = offset_scale
        self.center_feature_scale = center_feature_scale

        self.dw_conv = nn.Sequential(
            nn.Conv2d(
                channels,
                channels,
                kernel_size=dw_kernel_size,
                stride=1,
                padding=(dw_kernel_size - 1) // 2,
                groups=channels),
            build_norm_layer(
                channels,
                norm_layer,
                'channels_first',
                'channels_last'),
            build_act_layer(act_layer))
        self.offset = nn.Linear(
            channels,
            group * kernel_size * kernel_size * 2)
        self.mask = nn.Linear(
            channels,
            group * kernel_size * kernel_size)
        self.input_proj = nn.Linear(channels, channels)
        self.output_proj = nn.Linear(channels, channels)
        self._reset_parameters()

        if center_feature_scale:
            self.center_feature_scale_proj_weight = nn.Parameter(
                torch.zeros((group, channels), dtype=torch.float))
            self.center_feature_scale_proj_bias = nn.Parameter(
                torch.tensor(0.0, dtype=torch.float).view((1,)).repeat(group, ))
            self.center_feature_scale_module = CenterFeatureScaleModule()

    def _reset_parameters(self):
        constant_(self.offset.weight.data, 0.)
        constant_(self.offset.bias.data, 0.)
        constant_(self.mask.weight.data, 0.)
        constant_(self.mask.bias.data, 0.)
        xavier_uniform_(self.input_proj.weight.data)
        constant_(self.input_proj.bias.data, 0.)
        xavier_uniform_(self.output_proj.weight.data)
        constant_(self.output_proj.bias.data, 0.)

    def forward(self, input):
        """
        :param query                       (N, H, W, C)
        :return output                     (N, H, W, C)
        """
        N, H, W, _ = input.shape

        x = self.input_proj(input)
        x_proj = x
        dtype = x.dtype

        x1 = input.permute(0, 3, 1, 2)
        x1 = self.dw_conv(x1)
        offset = self.offset(x1)
        mask = self.mask(x1).reshape(N, H, W, self.group, -1)
        mask = F.softmax(mask, -1).reshape(N, H, W, -1).type(dtype)

        x = DCNv3Function.apply(
            x, offset, mask,
            self.kernel_size, self.kernel_size,
            self.stride, self.stride,
            self.pad, self.pad,
            self.dilation, self.dilation,
            self.group, self.group_channels,
            self.offset_scale,
            256)

        if self.center_feature_scale:
            center_feature_scale = self.center_feature_scale_module(
                x1, self.center_feature_scale_proj_weight, self.center_feature_scale_proj_bias)
            # N, H, W, groups -> N, H, W, groups, 1 -> N, H, W, groups, _d_per_group -> N, H, W, channels
            center_feature_scale = center_feature_scale[..., None].repeat(
                1, 1, 1, 1, self.channels // self.group).flatten(-2)
            x = x * (1 - center_feature_scale) + x_proj * center_feature_scale
        x = self.output_proj(x)

        return x
# --------------------------end----只是DCNv3模块---------------


class MLPLayer(nn.Module):
    r""" MLP layer of InternImage
    Args:
        in_features (int): number of input features
        hidden_features (int): number of hidden features
        out_features (int): number of output features
        act_layer (str): activation layer
        drop (float): dropout rate
    """

    def __init__(self,
                 in_features,
                 hidden_features=None,
                 out_features=None,
                 act_layer='GELU',
                 drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = build_act_layer(act_layer)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

## H×W×3 ---》 H×W×C
## 作为浅特征提取模块
class StemLayer(nn.Module):
    r""" Stem layer of InternImage
    Args:
        in_chans (int): number of input channels
        out_chans (int): number of output channels
        act_layer (str): activation layer
        norm_layer (str): normalization layer
    """

    def __init__(self,
                 in_chans=3,
                 out_chans=32,
                 act_layer='GELU',
                 norm_layer='BN'):
        super().__init__()
        self.conv1 = nn.Conv2d(in_chans,
                               out_chans // 2,
                               kernel_size=3,
                               stride=1,
                               padding=1)
        self.norm1 = build_norm_layer(out_chans // 2, norm_layer,
                                      'channels_first', 'channels_first')
        self.act = build_act_layer(act_layer)
        self.conv2 = nn.Conv2d(out_chans // 2,
                               out_chans,
                               kernel_size=3,
                               stride=1,
                               padding=1)
        self.norm2 = build_norm_layer(out_chans, norm_layer, 'channels_first',
                                      'channels_last')

    def forward(self, x):
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.act(x)
        x = self.conv2(x)
        x = self.norm2(x)
        return x


In [ ]:
# Output Projection
# B, C, H, W
class OutputProj(nn.Module):
    def __init__(self, in_channel=64, out_channel=3, kernel_size=3, stride=1, norm_layer=None,act_layer=None):
        super().__init__()
        self.proj = nn.Sequential(
            nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride, padding=kernel_size//2),
        )
        if act_layer is not None:
            self.proj.add_module(act_layer(inplace=True))
        if norm_layer is not None:
            self.norm = norm_layer(out_channel)
        else:
            self.norm = None
        self.in_channel = in_channel
        self.out_channel = out_channel

    def forward(self, x):
        x = x.permute(0,3,1,2)
        x = self.proj(x)
        if self.norm is not None:
            x = self.norm(x)
        return x

    def flops(self, H, W):
        flops = 0
        # conv
        flops += H*W*self.in_channel*self.out_channel*3*3

        if self.norm is not None:
            flops += H*W*self.out_channel
        print("Output_proj:{%.2f}"%(flops/1e9))
        return flops

In [ ]:
################ InternImage的downsample/upsample #########################
class DownsampleLayer(nn.Module):
    r""" Downsample layer of InternImage
    Args:
        channels (int): number of input channels
        norm_layer (str): normalization layer
    """
    def __init__(self, channels, norm_layer='LN'):
        super().__init__()
        self.conv = nn.Conv2d(channels,
                              2 * channels,
                              kernel_size=3,
                              stride=2,
                              padding=1,
                              bias=False)
        self.norm = build_norm_layer(2 * channels, norm_layer,
                                     'channels_first', 'channels_last')

    def forward(self, x):
        x = self.conv(x.permute(0, 3, 1, 2)) # channels_first
        x = self.norm(x)
        return x

##
class UpsampleLayer(nn.Module):
    def __init__(self, in_channels, out_channel, norm_layer='LN'):
        super().__init__()
        self.conv = nn.ConvTranspose2d(in_channels,
                              out_channel,
                              kernel_size=3,
                              stride=2,
                              padding=1,
                              output_padding=1,
                              bias=False)
        self.norm = build_norm_layer(out_channel, norm_layer,
                                     'channels_first', 'channels_last')

    def forward(self, x):
        x = self.conv(x.permute(0, 3, 1, 2)) # first_channel
        x = self.norm(x)

        return x
# downsample = DownsampleLayer(32)
# upsample = UpsampleLayer(64)
# img_down = downsample(x)
# img_up = upsample(img_down)
# img_down.shape,img_up.shape,x.shape

In [ ]:

## 模仿transformer架构，DCNv3替换注意力模块
class DCNv3Layer(nn.Module):
    r""" Basic layer of InternImage
    Args:
        core_op (nn.Module): core operation of InternImage
        channels (int): number of input channels
        groups (list): Groups of each block.
        mlp_ratio (float): ratio of mlp hidden features to input channels
        drop (float): dropout rate
        drop_path (float): drop path rate
        act_layer (str): activation layer
        norm_layer (str): normalization layer
        post_norm (bool): whether to use post normalization
        layer_scale (float): layer scale
        offset_scale (float): offset scale
        with_cp (bool): whether to use checkpoint
    """

    def __init__(self,
                 channels,
                 groups,
                 mlp_ratio=4.,
                 drop=0.,
                 drop_path=0.,
                 act_layer='GELU',
                 norm_layer='LN',
                 post_norm=False,
                 layer_scale=None,
                 offset_scale=1.0,
                 with_cp=False):
        super().__init__()
        self.channels = channels
        self.groups = groups
        self.mlp_ratio = mlp_ratio
        self.with_cp = with_cp

        self.norm1 = build_norm_layer(channels, 'LN')
        self.post_norm = post_norm
        self.dcn = DCNv3_pytorch(channels=channels,
                           kernel_size=3,
                           stride=1,
                           pad=1,
                           dilation=1,
                           group=groups,
                           offset_scale=offset_scale,
                           act_layer=act_layer,
                           norm_layer=norm_layer)
        self.drop_path = DropPath(drop_path) if drop_path > 0. \
            else nn.Identity()
        self.norm2 = build_norm_layer(channels, 'LN')
        self.mlp = MLPLayer(in_features=channels,
                            hidden_features=int(channels * mlp_ratio),
                            act_layer=act_layer,
                            drop=drop)
        self.layer_scale = layer_scale is not None
        if self.layer_scale:
            self.gamma1 = nn.Parameter(layer_scale * torch.ones(channels),
                                       requires_grad=True)
            self.gamma2 = nn.Parameter(layer_scale * torch.ones(channels),
                                       requires_grad=True)

    def forward(self, x):

        def _inner_forward(x):
            if not self.layer_scale:
                if self.post_norm:
                    x = x + self.drop_path(self.norm1(self.dcn(x)))
                    x = x + self.drop_path(self.norm2(self.mlp(x)))
                else:
                    x = x + self.drop_path(self.dcn(self.norm1(x)))
                    x = x + self.drop_path(self.mlp(self.norm2(x)))
                return x
            if self.post_norm:
                x = x + self.drop_path(self.gamma1 * self.norm1(self.dcn(x)))
                x = x + self.drop_path(self.gamma2 * self.norm2(self.mlp(x)))
            else:
                x = x + self.drop_path(self.gamma1 * self.dcn(self.norm1(x)))
                x = x + self.drop_path(self.gamma2 * self.mlp(self.norm2(x)))
            return x

        if self.with_cp and x.requires_grad:
            x = checkpoint.checkpoint(_inner_forward, x)
        else:
            x = _inner_forward(x)
        return x


class DCNv3Block(nn.Module):

    def __init__(self,
                 channels,
                 depth,
                 groups,
                 mlp_ratio=4.,
                 drop=0.,
                 drop_path=0.,
                 act_layer='GELU',
                 norm_layer='LN',
                 post_norm=False,
                 offset_scale=1.0,
                 layer_scale=None,
                 with_cp=False):
        super().__init__()
        self.channels = channels
        self.depth = depth 
        self.post_norm = post_norm

        self.blocks = nn.ModuleList([
            DCNv3Layer(channels=channels,
                             groups=groups,
                             mlp_ratio=mlp_ratio,
                             drop=drop,
                             drop_path=drop_path[i] if isinstance(drop_path, list) else drop_path,
                             act_layer=act_layer,
                             norm_layer=norm_layer,
                             post_norm=post_norm,
                             layer_scale=layer_scale,
                             offset_scale=offset_scale,
                             with_cp=with_cp) for i in range(depth)
        ])
        if not self.post_norm:
            self.norm = build_norm_layer(channels, 'LN')

    def forward(self, x):
        '''
        :param x:
        :param return_wo_downsample: 这个参数的作用是为了在需要对模型的中间结果进行分析或可视化时，方便地获取模型的所有层的输出。
        :return:
        '''
        # 将输入张量 x 通过 self.blocks 中的所有块进行前向传递，得到输出张量 x
        for blk in self.blocks:
            x = blk(x)
        if not self.post_norm:
            x = self.norm(x)
        return x

'''
----------------end------------------
'''

x = torch.randn(1, 28, 28, 3).permute(0, 3, 1, 2)
embed_dim = 32
mlp_ratio=4.
drop_rate=0.
drop_path_rate=0.1
depths=[2, 2, 2, 2, 2, 2, 2, 2, 2]
num_enc_layers = len(depths)//2
num_dec_layers = len(depths)//2
enc_dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths[:num_enc_layers]))]
conv_dpr = [drop_path_rate]*depths[4]
dec_dpr = enc_dpr[::-1]
input = StemLayer(3, embed_dim, 'GELU', 'BN')
layer = DCNv3Layer(
    embed_dim, 2, mlp_ratio, drop_rate, act_layer='GELU',
            norm_layer='LN',
            post_norm=False,
            layer_scale=None,
            offset_scale=1.5,
            with_cp=False
)
block = DCNv3Block(channels=embed_dim,
            depth=2,
            groups=2,
            mlp_ratio=mlp_ratio,
            drop=drop_rate,  # drop = []
            drop_path=enc_dpr[sum(depths[:0]):sum(depths[:1])],
            act_layer='GELU',
            norm_layer='LN',
            post_norm=False,
            layer_scale=None,
            offset_scale=1.5,
            with_cp=False)
x = input(x)
x.shape
x = layer(x)
x = layer(x)
x.shape
print(block(x).shape)

torch.Size([1, 28, 28, 32])


In [ ]:
class Uformer(nn.Module):
    r'''
        channels,
        depth,
        groups,
        downsample=True,
        mlp_ratio=4.,
        drop=0.,
        drop_path=0.,
        act_layer='GELU',
        norm_layer='LN',
        post_norm=False,
        offset_scale=1.0,
        layer_scale=None,
        with_cp=False
    '''
    def __init__(self,
                 # img_size=256,
                 in_chans=3,
                 dd_in=3,
                 embed_dim=32,
                 depths=[2, 2, 2, 2, 2, 2, 2, 2, 2],
                 # num_heads=[1, 2, 4, 8, 16, 16, 8, 4, 2], # groups
                 # win_size=8,
                 mlp_ratio=4.,
                 # qkv_bias=True,
                 # qk_scale=None,
                 drop_rate=0.,
                 # attn_drop_rate=0.,
                 drop_path_rate=0.1,
                 # norm_layer=nn.LayerNorm,
                 patch_norm=True,
                 use_checkpoint=False,
                 token_projection='linear',
                 token_mlp='leff',
                 dowsample=DownsampleLayer,
                 upsample=UpsampleLayer,
                 # shift_flag=True,
                 # modulator=False,
                 # cross_modulator=False, **kwargs
                 ):
        super().__init__()

        self.num_enc_layers = len(depths)//2
        self.num_dec_layers = len(depths)//2
        self.embed_dim = embed_dim
        self.patch_norm = patch_norm
        self.mlp_ratio = mlp_ratio
        self.token_projection = token_projection
        self.mlp = token_mlp
        # self.win_size =win_size
        # self.reso = img_size
        self.pos_drop = nn.Dropout(p=drop_rate)
        self.dd_in = dd_in

        # stochastic depth
        # dropout率
        enc_dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths[:self.num_enc_layers]))]
        conv_dpr = [drop_path_rate]*depths[4]
        dec_dpr = enc_dpr[::-1]

        # build layers

        # Input/Output
        self.input_proj = StemLayer(3, embed_dim, 'GELU', 'BN') ## B, C, H, W
        self.output_proj = OutputProj(in_channel=2*embed_dim, out_channel=in_chans, kernel_size=3, stride=1)

        # Encoder
        self.en_layer0 = DCNv3Block(channels=embed_dim,
                             depth=depths[0],
                             groups=2,
                             mlp_ratio=mlp_ratio,
                             drop=drop_rate,  # drop = []
                             drop_path=enc_dpr[sum(depths[:0]):sum(depths[:1])],
                             act_layer='GELU',
                             norm_layer='LN',
                             post_norm=False,
                             layer_scale=None,
                             offset_scale=1.5,
                             with_cp=False)
        self.dowsample_0 = dowsample(embed_dim)
        self.en_layer1 = DCNv3Block(channels=embed_dim*2,
                                    depth=depths[1],
                                    groups=2,
                                    mlp_ratio=mlp_ratio,
                                    drop=drop_rate,  # drop = []
                                    drop_path=enc_dpr[sum(depths[:0]):sum(depths[:1])],
                                    act_layer='GELU',
                                    norm_layer='LN',
                                    post_norm=False,
                                    layer_scale=None,
                                    offset_scale=1.5,
                                    with_cp=False)
        self.dowsample_1 = dowsample(embed_dim*2)
        self.en_layer2 = DCNv3Block(channels=embed_dim * 4,
                                    depth=depths[2],
                                    groups=2,
                                    mlp_ratio=mlp_ratio,
                                    drop=drop_rate,  # drop = []
                                    drop_path=enc_dpr[sum(depths[:0]):sum(depths[:1])],
                                    act_layer='GELU',
                                    norm_layer='LN',
                                    post_norm=False,
                                    layer_scale=None,
                                    offset_scale=1.5,
                                    with_cp=False)
        self.dowsample_2 = dowsample(embed_dim*4)
        self.en_layer3 = DCNv3Block(channels=embed_dim * 8,
                                    depth=depths[3],
                                    groups=2,
                                    mlp_ratio=mlp_ratio,
                                    drop=drop_rate,  # drop = []
                                    drop_path=enc_dpr[sum(depths[:0]):sum(depths[:1])],
                                    act_layer='GELU',
                                    norm_layer='LN',
                                    post_norm=False,
                                    layer_scale=None,
                                    offset_scale=1.5,
                                    with_cp=False)
        self.dowsample_3 = dowsample(embed_dim*8)

        # Bottleneck
        self.conv = DCNv3Block(channels=embed_dim * 16,
                                    depth=depths[4],
                                    groups=2,
                                    mlp_ratio=mlp_ratio,
                                    drop=drop_rate,  # drop = []
                                    drop_path=enc_dpr[sum(depths[:0]):sum(depths[:1])],
                                    act_layer='GELU',
                                    norm_layer='LN',
                                    post_norm=False,
                                    layer_scale=None,
                                    offset_scale=1.5,
                                    with_cp=False)

        # Decoder
        self.upsample_0 = upsample(embed_dim*16, embed_dim*8)
        self.de_layer0 = DCNv3Block(channels=embed_dim * 16,
                                    depth=depths[5],
                                    groups=2,
                                    mlp_ratio=mlp_ratio,
                                    drop=drop_rate,  # drop = []
                                    drop_path=enc_dpr[sum(depths[:0]):sum(depths[:1])],
                                    act_layer='GELU',
                                    norm_layer='LN',
                                    post_norm=False,
                                    layer_scale=None,
                                    offset_scale=1.5,
                                    with_cp=False)
        self.upsample_1 = upsample(embed_dim*16, embed_dim*4)
        self.de_layer1 = DCNv3Block(channels=embed_dim * 8,
                                    depth=depths[6],
                                    groups=2,
                                    mlp_ratio=mlp_ratio,
                                    drop=drop_rate,  # drop = []
                                    drop_path=enc_dpr[sum(depths[:0]):sum(depths[:1])],
                                    act_layer='GELU',
                                    norm_layer='LN',
                                    post_norm=False,
                                    layer_scale=None,
                                    offset_scale=1.5,
                                    with_cp=False)
        self.upsample_2 = upsample(embed_dim*8, embed_dim*2)
        self.de_layer2 = DCNv3Block(channels=embed_dim * 4,
                                    depth=depths[7],
                                    groups=2,
                                    mlp_ratio=mlp_ratio,
                                    drop=drop_rate,  # drop = []
                                    drop_path=enc_dpr[sum(depths[:0]):sum(depths[:1])],
                                    act_layer='GELU',
                                    norm_layer='LN',
                                    post_norm=False,
                                    layer_scale=None,
                                    offset_scale=1.5,
                                    with_cp=False)
        self.upsample_3 = upsample(embed_dim*4, embed_dim)
        self.de_layer3 = DCNv3Block(channels=embed_dim * 2,
                                    depth=depths[8],
                                    groups=2,
                                    mlp_ratio=mlp_ratio,
                                    drop=drop_rate,  # drop = []
                                    drop_path=enc_dpr[sum(depths[:0]):sum(depths[:1])],
                                    act_layer='GELU',
                                    norm_layer='LN',
                                    post_norm=False,
                                    layer_scale=None,
                                    offset_scale=1.5,
                                    with_cp=False)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.jit.ignore
    def no_weight_decay(self):
        return {'absolute_pos_embed'}

    @torch.jit.ignore
    def no_weight_decay_keywords(self):
        return {'relative_position_bias_table'}

    def extra_repr(self) -> str:
        return f"embed_dim={self.embed_dim}, token_projection={self.token_projection}, token_mlp={self.mlp}"

    def forward(self, x, mask=None):
        # Input Projection
        y = self.input_proj(x)
        y = self.pos_drop(y)
        #Encoder
        # conv0 = self.encoderlayer_0(y,mask=mask)
        print(y.shape)
        conv0 = self.en_layer0(y)
        # print(conv0.shape)
        pool0 = self.dowsample_0(conv0)
        # print(pool0.shape)
        # conv1 = self.encoderlayer_1(pool0,mask=mask)
        conv1 = self.en_layer1(pool0)
        pool1 = self.dowsample_1(conv1)
        # print(pool1.shape)
        # conv2 = self.encoderlayer_2(pool1,mask=mask)
        conv2 = self.en_layer2(pool1)
        pool2 = self.dowsample_2(conv2)
        # print(pool2.shape)
        # conv3 = self.encoderlayer_3(pool2,mask=mask)
        conv3 = self.en_layer3(pool2)
        # print(conv3.shape)
        pool3 = self.dowsample_3(conv3)
        # print(pool3.shape)

        # Bottleneck
        # conv4 = self.conv(pool3, mask=mask)
        conv4 = self.conv(pool3)

        #Decoder
        up0 = self.upsample_0(conv4)
        # print(up0.shape)

        # skip connection
        deconv0 = torch.cat([up0,conv3],-1)
        # print(deconv0.shape)

        deconv0 = self.de_layer0(deconv0)
        # print(deconv0.shape)

        up1 = self.upsample_1(deconv0)
        # print(up1.shape)
        deconv1 = torch.cat([up1,conv2],-1)
        # deconv1 = self.decoderlayer_1(deconv1,mask=mask)
        # print(deconv1.shape)
        deconv1 = self.de_layer1(deconv1)

        up2 = self.upsample_2(deconv1)
        deconv2 = torch.cat([up2,conv1],-1)
        # deconv2 = self.decoderlayer_2(deconv2,mask=mask)
        deconv2 = self.de_layer2(deconv2)

        up3 = self.upsample_3(deconv2)
        deconv3 = torch.cat([up3,conv0],-1)
        # deconv3 = self.decoderlayer_3(deconv3,mask=mask)
        deconv3 = self.de_layer3(deconv3)
        print(deconv3.shape)

        # Output Projection
        y = self.output_proj(deconv3)
        return x + y if self.dd_in ==3 else y

    # def flops(self):
    #     flops = 0
    #     # Input Projection
    #     flops += self.input_proj.flops(self.reso,self.reso)
    #     # Encoder
    #     flops += self.en_layer0.flops()+self.dowsample_0.flops(self.reso,self.reso)
    #     flops += self.en_layer1.flops()+self.dowsample_1.flops(self.reso//2,self.reso//2)
    #     flops += self.en_layer2.flops()+self.dowsample_2.flops(self.reso//2**2,self.reso//2**2)
    #     flops += self.en_layer3.flops()+self.dowsample_3.flops(self.reso//2**3,self.reso//2**3)

    #     # Bottleneck
    #     flops += self.conv.flops()

    #     # Decoder
    #     flops += self.upsample_0.flops(self.reso//2**4,self.reso//2**4)+self.de_layer0.flops()
    #     flops += self.upsample_1.flops(self.reso//2**3,self.reso//2**3)+self.de_layer1.flops()
    #     flops += self.upsample_2.flops(self.reso//2**2,self.reso//2**2)+self.de_layer2.flops()
    #     flops += self.upsample_3.flops(self.reso//2,self.reso//2)+self.de_layer3.flops()

    #     # Output Projection
    #     flops += self.output_proj.flops(self.reso,self.reso)
    #     return flops


input_size = 256
depths=[2, 2, 2, 2, 2, 2, 2, 2, 2]
# model_restoration = Uformer(img_size=input_size, embed_dim=16,depths=depths,
#              win_size=8, mlp_ratio=4., token_projection='linear', token_mlp='leff', modulator=True, shift_flag=False)
model_restoration = Uformer(in_chans=3, dd_in=3, embed_dim= 16, depths=depths)
x = torch.randn(1, 256, 256, 3).permute(0, 3, 1, 2) # first_channel
model_restoration(x).shape

torch.Size([1, 256, 256, 16])
torch.Size([1, 256, 256, 32])


torch.Size([1, 3, 256, 256])